<a href="https://colab.research.google.com/github/mazen200/AI_Tutorial/blob/main/useTopics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

## Importing libiraries

In [161]:
!pip install vaderSentiment
!pip install category_encoders

In [162]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,f1_score,recall_score,precision_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import pickle
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier

## load data set

In [163]:
data = pd.read_csv("/content/sentimentdataset.csv")

In [164]:
data.head()

,ID,Text,Sentiment (Label),Timestamp,User,Source,Topic,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,Enjoying a beautiful day at the park! ...,Positive,1/15/2023 12:30,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,1,Traffic was terrible this morning. ...,Negative,1/15/2023 8:45,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,2,Just finished an amazing workout! 💪 ...,Positive,1/15/2023 15:45,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,3,Excited about the upcoming weekend getaway! ...,Positive,1/15/2023 18:20,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,4,Trying out a new recipe for dinner tonight. ...,Neutral,1/15/2023 19:55,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


In [165]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Preprocessing

- Remove punctuation and stopwords, and convert the string to lowercase.



In [166]:
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

- Remove emojis function

In [167]:
import re

def remove_emojis(text):
    # Define a regular expression pattern to match emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Remove emojis from the text using the pattern
    return emoji_pattern.sub(r'', text)

In [168]:
#data['Text'] = data['Text'].apply(remove_emojis)
data['Text'] = data['Text'].apply(preprocess_text)

In [169]:
data.head(10)

,ID,Text,Sentiment (Label),Timestamp,User,Source,Topic,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,Enjoying beautiful day park,Positive,1/15/2023 12:30,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,1,Traffic terrible morning,Negative,1/15/2023 8:45,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,2,finished amazing workout 💪,Positive,1/15/2023 15:45,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,3,Excited upcoming weekend getaway,Positive,1/15/2023 18:20,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,4,Trying new recipe dinner tonight,Neutral,1/15/2023 19:55,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19
5,5,Feeling grateful little thing life,Positive,1/16/2023 9:10,GratitudeNow,Twitter,#Gratitude #PositiveVibes,25,50,India,2023,1,16,9
6,6,Rainy day call cozy blanket hot cocoa,Positive,1/16/2023 14:45,RainyDays,Facebook,#RainyDays #Cozy,10,20,Canada,2023,1,16,14
7,7,new movie release mustwatch,Positive,1/16/2023 19:30,MovieBuff,Instagram,#MovieNight #MustWatch,15,30,USA,2023,1,16,19
8,8,Political discussion heating timeline,Negative,1/17/2023 8:00,DebateTalk,Twitter,#Politics #Debate,30,60,USA,2023,1,17,8
9,9,Missing summer vibe beach day,Neutral,1/17/2023 12:20,BeachLover,Facebook,#Summer #BeachDays,18,35,Australia,2023,1,17,12


In [170]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 732 non-null    int64 
 1   Text               732 non-null    object
 2   Sentiment (Label)  732 non-null    object
 3   Timestamp          732 non-null    object
 4   User               732 non-null    object
 5   Source             732 non-null    object
 6   Topic              732 non-null    object
 7   Retweets           732 non-null    int64 
 8   Likes              732 non-null    int64 
 9   Country            732 non-null    object
 10  Year               732 non-null    int64 
 11  Month              732 non-null    int64 
 12  Day                732 non-null    int64 
 13  Hour               732 non-null    int64 
dtypes: int64(7), object(7)
memory usage: 80.2+ KB


In [171]:
data.drop(columns=['User','Timestamp','Source','Retweets',	'Likes','ID',	'Country','Year','Month',	'Day','Hour'], inplace=True)

In [172]:
data.describe()

,Text,Sentiment (Label),Topic
count,732,732,732
unique,706,279,697
top,compassionate rain tear empathy falling gently...,Positive,#Compassionate #TearsOfEmpathy
freq,3,44,3


- split topic column to two columns

In [173]:
data['Topic'] = data['Topic'].str.replace('#', '')
data['Topic'] = data['Topic'].str.strip()
data[['Topic1', 'Topic2']] = data['Topic'].str.split(' ', expand=True)
data.drop(columns=['Topic'],inplace=True)

In [174]:
data.head()

,Text,Sentiment (Label),Topic1,Topic2
0,Enjoying beautiful day park,Positive,Nature,Park
1,Traffic terrible morning,Negative,Traffic,Morning
2,finished amazing workout 💪,Positive,Fitness,Workout
3,Excited upcoming weekend getaway,Positive,Travel,Adventure
4,Trying new recipe dinner tonight,Neutral,Cooking,Food


In [175]:
data.describe()

,Text,Sentiment (Label),Topic1,Topic2
count,732,732,732,732
unique,706,279,372,636
top,compassionate rain tear empathy falling gently...,Positive,Serenity,HighSchoolArt
freq,3,44,15,4


- make all label data only positive,negative and neutral.

In [176]:
analyzer = SentimentIntensityAnalyzer()
def classify_sentiment(label):
    sentiment_score = analyzer.polarity_scores(label)
    if sentiment_score['compound'] >= 0.05:
        return 'positive'
    elif sentiment_score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [177]:
data['Sentiment (Label)'] = data['Sentiment (Label)'].apply(classify_sentiment)

In [178]:
data.describe()

,Text,Sentiment (Label),Topic1,Topic2
count,732,732,732,732
unique,706,3,372,636
top,compassionate rain tear empathy falling gently...,positive,Serenity,HighSchoolArt
freq,3,378,15,4


- Replace label column with integer.

In [179]:
def REPLACE(label):
    if label == 'positive':
        return 1
    elif label == 'negative':
        return -1
    else:
        return 0

In [180]:
data['Sentiment (Label)'] = data['Sentiment (Label)'].apply(REPLACE)

In [181]:
data.head(10)

,Text,Sentiment (Label),Topic1,Topic2
0,Enjoying beautiful day park,1,Nature,Park
1,Traffic terrible morning,-1,Traffic,Morning
2,finished amazing workout 💪,1,Fitness,Workout
3,Excited upcoming weekend getaway,1,Travel,Adventure
4,Trying new recipe dinner tonight,0,Cooking,Food
5,Feeling grateful little thing life,1,Gratitude,PositiveVibes
6,Rainy day call cozy blanket hot cocoa,1,RainyDays,Cozy
7,new movie release mustwatch,1,MovieNight,MustWatch
8,Political discussion heating timeline,-1,Politics,Debate
9,Missing summer vibe beach day,0,Summer,BeachDays


## Feature Engineering

In [182]:
vectorizer = TfidfVectorizer()
# Fit the vectorizer to the text data and transform it into TF-IDF vectors
tfidf_vectors = vectorizer.fit_transform(data['Text'])
# Convert the TF-IDF vectors into a DataFrame
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())
# Save the DataFrame with TF-IDF vectors to a CSV file
tfidf_df.to_csv('tfidf_vectorsText.csv',index=False)

In [183]:
tfidf_vectors = vectorizer.fit_transform(data['Topic1'])

# Convert the TF-IDF vectors into a DataFrame
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Save the DataFrame with TF-IDF vectors to a CSV file
tfidf_df.to_csv('Topic1_vectorsText.csv',index=False)

In [184]:
tfidf_vectors = vectorizer.fit_transform(data['Topic2'])

# Convert the TF-IDF vectors into a DataFrame
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Save the DataFrame with TF-IDF vectors to a CSV file
tfidf_df.to_csv('Topic2_vectorsText.csv',index=False)

In [185]:
# Load the three CSV files
df1 = pd.read_csv('tfidf_vectorsText.csv')
df2 = pd.read_csv('Topic1_vectorsText.csv')
df3 = pd.read_csv('Topic2_vectorsText.csv')
# Concatenate the DataFrames vertically
merged_df = pd.concat([df1, df2, df3], ignore_index=True, axis = 1)
merged_df['Label'] = data['Sentiment (Label)']
# Write the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)

In [186]:
merged_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,3308,3309,3310,3311,3312,3313,3314,3315,3316,Label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [187]:
df = pd.read_csv('merged_file.csv')
# Step 2: Separate features (X) from labels (y)
X = df.drop(columns=['Label'])  # Assuming 'label_column' is the name of the column containing labels
y = df['Label']

In [188]:
X.head(10)

,0,1,2,3,4,5,6,7,8,9,...,3307,3308,3309,3310,3311,3312,3313,3314,3315,3316
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Model Evaluation

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [193]:
model = SVC(kernel = 'linear',C=0.3)
model.fit(X_train, y_train)

SVC(C=0.3, kernel='linear')

In [194]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred,average ='weighted')
recall = recall_score(y_test, y_pred,average ='weighted')
f1sc = f1_score(y_test, y_pred,average ='weighted')

train_acc = accuracy_score(y_train,model.predict(X_train))
print("Training Accuracy:", train_acc)
# Cross-validation
cv_acc = cross_val_score(model, X_train, y_train, cv=5)
print("Cross-validation Accuracies:", cv_acc)

print("test Accuracy:", accuracy)
print("precision : " , precision)
print("recall : ", recall)
print("f1 score : ",f1sc)

Training Accuracy: 0.9963570127504554
Cross-validation Accuracies: [0.77272727 0.8        0.75454545 0.76363636 0.78899083]
test Accuracy: 0.7540983606557377
precision :  0.8320408439292183
recall :  0.7540983606557377
f1 score :  0.7370868389740691
